# Trabalho Praticas Parte 03

In [1]:
import pandas as pd
from datetime import datetime,date
from pandasql import sqldf
from life.insurance import isrc_discrete

In [2]:
aposentados = pd.read_csv('APOSENTADOS.csv',delimiter=";")

In [3]:
aposentados.head()

,Unnamed: 0,NU_ANO,NU_MES,CO_IBGE,NO_ENTE,SG_UF,CO_COMP_MASSA,CO_TIPO_FUNDO,CNPJ_ORGAO,NO_ORGAO,...,VL_CONTRIBUICAO,VL_COMPENS_PREVID,IN_PARID_SERV,CO_CONDICAO_APOSENTADO,NU_DEPENDENTES,NU_TEMPO_RPPS_MUN,NU_TEMPO_RPPS_EST,NU_TEMPO_RPPS_FED,IN_PREV_COMP,VL_TETO_ESPECIFICO
0,0,2021,12,2306504,Itapiúna,Ceará,1,1,3333333,Instituto,...,0.0,0,1,1,1,0,0,0,2,29000
1,1,2021,12,2306504,Itapiúna,Ceará,1,1,3333333,Instituto,...,0.0,0,1,1,1,0,0,0,2,29000
2,2,2021,12,2306504,Itapiúna,Ceará,1,1,3333333,Instituto,...,0.0,0,2,1,1,0,0,0,2,29000
3,3,2021,12,2306504,Itapiúna,Ceará,1,1,3333333,Instituto,...,0.0,0,1,1,1,0,0,0,2,29000
4,4,2021,12,2306504,Itapiúna,Ceará,1,1,3333333,Instituto,...,0.0,0,1,1,1,0,0,0,2,29000


In [4]:
aposentados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6782 entries, 0 to 6781
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               6782 non-null   int64  
 1   NU_ANO                   6782 non-null   int64  
 2   NU_MES                   6782 non-null   int64  
 3   CO_IBGE                  6782 non-null   int64  
 4   NO_ENTE                  6782 non-null   object 
 5   SG_UF                    6782 non-null   object 
 6   CO_COMP_MASSA            6782 non-null   int64  
 7   CO_TIPO_FUNDO            6782 non-null   int64  
 8   CNPJ_ORGAO               6782 non-null   int64  
 9   NO_ORGAO                 6782 non-null   object 
 10  CO_PODER                 6782 non-null   int64  
 11  CO_TIPO_PODER            6782 non-null   int64  
 12  CO_TIPO_POPULACAO        6782 non-null   int64  
 13  CO_TIPO_CARGO            6782 non-null   int64  
 14  CO_TIPO_APOSENTADORIA   

In [5]:
aposentados['DT_NASC_APOSENTADO'] = pd.to_datetime(aposentados['DT_NASC_APOSENTADO'])

In [6]:
today_date =  datetime.now()
today_date = today_date.strftime( "%d-%m-%Y")
today_date = pd.to_datetime(today_date)

In [7]:
def age(born):
    today = date.today()
    return today.year - born.year - ((today.month, 
                                      today.day) < (born.month, 
                                                    born.day))

In [8]:
aposentados['IDADE'] = aposentados['DT_NASC_APOSENTADO'].apply(age)

### Criando Vetores de Anuidades 

In [9]:
lt = pd.read_csv('life/life_tables.csv')

In [10]:
lt.tail()

,AGE,AT_2000_F,AT_2000_M,AT_49_F,AT_49_M,AT_83_F,AT_83_M,IBGE_2020_F,IBGE_2020_M
111,111,0.644427,0.668186,1.0,1.0,0.650646,0.695704,1.0,1.0
112,112,0.719484,0.739483,1.0,1.0,0.724750,0.762343,1.0,1.0
113,113,0.803380,0.818254,1.0,1.0,0.807316,0.835056,1.0,1.0
114,114,0.896693,0.904945,1.0,1.0,0.898885,0.914167,1.0,1.0
115,115,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0


In [11]:
from life.annuity import ann_discrete 

ax_discrete_f_v = ann_discrete(table=lt['IBGE_2020_F'])
ax_discrete_m_v = ann_discrete(table=lt['IBGE_2020_M'])
ax_discrete_f_inv = ann_discrete(table=lt['AT_2000_F'])
ax_discrete_m_inv = ann_discrete(table=lt['AT_2000_M'])

In [12]:
idades = aposentados['IDADE'].unique()
axs= pd.DataFrame(columns=['IDADE','AX_FEM_VALIDOS','AX_MASC_VALIDOS','AX_FEM_INVALIDOS','AX_MASC_INVALIDOS'])

In [13]:
for i in range(len(idades)):
    ax_f_v = ax_discrete_f_v.ax(i=0.05, age=idades[i], B=1, due=True)
    ax_m_v = ax_discrete_m_v.ax(i=0.05, age=idades[i], B=1, due=True)
    ax_m_inv = ax_discrete_f_inv.ax(i=0.05, age=idades[i], B=1, due=True)
    ax_f_inv = ax_discrete_m_inv.ax(i=0.05, age=idades[i], B=1, due=True)
    axs = pd.DataFrame([[idades[i], ax_f_v,ax_m_v,ax_m_inv,ax_f_inv]],columns = axs.columns).append(axs)


In [14]:
axs.head()

,IDADE,AX_FEM_VALIDOS,AX_MASC_VALIDOS,AX_FEM_INVALIDOS,AX_MASC_INVALIDOS
0,93,4.673620,3.863685,4.422309,4.306042
0,92,4.930574,4.116336,4.634489,4.503990
0,96,3.902483,3.098820,3.868830,3.758676
0,91,5.187882,4.367831,4.863253,4.711026
0,85,6.757088,5.860302,6.580792,6.153596


## Calculo do valor dos Benefícios dos Aposentados

In [15]:
aposentados.columns

Index(['Unnamed: 0', 'NU_ANO', 'NU_MES', 'CO_IBGE', 'NO_ENTE', 'SG_UF',
       'CO_COMP_MASSA', 'CO_TIPO_FUNDO', 'CNPJ_ORGAO', 'NO_ORGAO', 'CO_PODER',
       'CO_TIPO_PODER', 'CO_TIPO_POPULACAO', 'CO_TIPO_CARGO',
       'CO_TIPO_APOSENTADORIA', 'ID_APOSENTADO_MATRICULA', 'ID_APOSENTADO_CPF',
       'ID_APOSENT_PIS_PASEP', 'CO_SEXO_APOSENTADO', 'CO_EST_CIVIL_APOSENTADO',
       'DT_NASC_APOSENTADO', 'DT_ING_SERV_PUB', 'DT_ING_ENTE',
       'DT_INICIO_APOSENTADORIA', 'VL_APOSENTADORIA', 'VL_CONTRIBUICAO',
       'VL_COMPENS_PREVID', 'IN_PARID_SERV', 'CO_CONDICAO_APOSENTADO',
       'NU_DEPENDENTES', 'NU_TEMPO_RPPS_MUN', 'NU_TEMPO_RPPS_EST',
       'NU_TEMPO_RPPS_FED', 'IN_PREV_COMP', 'VL_TETO_ESPECIFICO', 'IDADE'],
      dtype='object')

In [16]:
vpbf_aposent = pd.merge(aposentados[['IDADE','CO_SEXO_APOSENTADO','CO_CONDICAO_APOSENTADO','VL_APOSENTADORIA']],
                        axs[['IDADE','AX_FEM_VALIDOS','AX_MASC_VALIDOS','AX_FEM_INVALIDOS','AX_MASC_INVALIDOS']],
                        on='IDADE',
                        how='inner')

In [17]:
vpbf_aposent.head()

,IDADE,CO_SEXO_APOSENTADO,CO_CONDICAO_APOSENTADO,VL_APOSENTADORIA,AX_FEM_VALIDOS,AX_MASC_VALIDOS,AX_FEM_INVALIDOS,AX_MASC_INVALIDOS
0,65,2,1,8972.34,12.595562,11.280892,13.336076,12.278015
1,65,2,1,8295.33,12.595562,11.280892,13.336076,12.278015
2,65,1,1,9716.90,12.595562,11.280892,13.336076,12.278015
3,65,1,1,4222.41,12.595562,11.280892,13.336076,12.278015
4,65,1,1,8577.66,12.595562,11.280892,13.336076,12.278015


In [18]:
p = 13
vpbf_aposent['VALOR_BENF_FUT'] = 0

In [19]:
for i in range(len(vpbf_aposent)):
   if vpbf_aposent['CO_SEXO_APOSENTADO'][i] == 1 and vpbf_aposent['CO_CONDICAO_APOSENTADO'][i] == 1:
      vpbf_aposent['VALOR_BENF_FUT'][i] = vpbf_aposent['AX_FEM_VALIDOS'][i] * vpbf_aposent['VL_APOSENTADORIA'][i] * p
      
   elif vpbf_aposent['CO_SEXO_APOSENTADO'][i] == 2 and vpbf_aposent['CO_CONDICAO_APOSENTADO'][i] == 1:
      vpbf_aposent['VALOR_BENF_FUT'][i] = vpbf_aposent['AX_MASC_VALIDOS'][i] * vpbf_aposent['VL_APOSENTADORIA'][i] * p

   elif vpbf_aposent['CO_SEXO_APOSENTADO'][i] == 1 and vpbf_aposent['CO_CONDICAO_APOSENTADO'][i] == 2:
      vpbf_aposent['VALOR_BENF_FUT'][i] = vpbf_aposent['AX_FEM_INVALIDOS'][i] * vpbf_aposent['VL_APOSENTADORIA'][i] * p

   elif vpbf_aposent['CO_SEXO_APOSENTADO'][i] == 2 and vpbf_aposent['CO_CONDICAO_APOSENTADO'][i] == 2:
      vpbf_aposent['VALOR_BENF_FUT'][i] = vpbf_aposent['AX_MASC_INVALIDOS'][i] * vpbf_aposent['VL_APOSENTADORIA'][i] * p

<ipython-input-19-b50e1364238a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vpbf_aposent['VALOR_BENF_FUT'][i] = vpbf_aposent['AX_MASC_VALIDOS'][i] * vpbf_aposent['VL_APOSENTADORIA'][i] * p
<ipython-input-19-b50e1364238a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vpbf_aposent['VALOR_BENF_FUT'][i] = vpbf_aposent['AX_FEM_VALIDOS'][i] * vpbf_aposent['VL_APOSENTADORIA'][i] * p


In [20]:
vpbf_aposent.head()

,IDADE,CO_SEXO_APOSENTADO,CO_CONDICAO_APOSENTADO,VL_APOSENTADORIA,AX_FEM_VALIDOS,AX_MASC_VALIDOS,AX_FEM_INVALIDOS,AX_MASC_INVALIDOS,VALOR_BENF_FUT
0,65,2,1,8972.34,12.595562,11.280892,13.336076,12.278015,1315808
1,65,2,1,8295.33,12.595562,11.280892,13.336076,12.278015,1216523
2,65,1,1,9716.90,12.595562,11.280892,13.336076,12.278015,1591067
3,65,1,1,4222.41,12.595562,11.280892,13.336076,12.278015,691387
4,65,1,1,8577.66,12.595562,11.280892,13.336076,12.278015,1404525


In [21]:
vpbf_aposent['VALOR_BENF_FUT'].sum()

10055467524

## Segunda Linha - Contribuição dos Aposentados 

In [22]:
vpcf_aposent = vpbf_aposent.copy()

In [23]:
vpcf_aposent.head()

,IDADE,CO_SEXO_APOSENTADO,CO_CONDICAO_APOSENTADO,VL_APOSENTADORIA,AX_FEM_VALIDOS,AX_MASC_VALIDOS,AX_FEM_INVALIDOS,AX_MASC_INVALIDOS,VALOR_BENF_FUT
0,65,2,1,8972.34,12.595562,11.280892,13.336076,12.278015,1315808
1,65,2,1,8295.33,12.595562,11.280892,13.336076,12.278015,1216523
2,65,1,1,9716.90,12.595562,11.280892,13.336076,12.278015,1591067
3,65,1,1,4222.41,12.595562,11.280892,13.336076,12.278015,691387
4,65,1,1,8577.66,12.595562,11.280892,13.336076,12.278015,1404525


In [24]:
vpcf_aposent['VALOR_CONTRIB_FUT'] = 0
t = 0.14
teto = 7087.22
p = 1

In [25]:
for i in range(len(vpbf_aposent)):
  if vpcf_aposent['VL_APOSENTADORIA'][i] > teto:
    if vpcf_aposent['CO_SEXO_APOSENTADO'][i] == 1 and vpcf_aposent['CO_CONDICAO_APOSENTADO'][i] == 1:
      vpcf_aposent['VALOR_CONTRIB_FUT'][i] = p * (vpcf_aposent['AX_FEM_VALIDOS'][i] * vpcf_aposent['VL_APOSENTADORIA'][i]) * ((vpcf_aposent['VL_APOSENTADORIA'][i] - teto ) / vpcf_aposent['VL_APOSENTADORIA'][i]) * t
      
    elif vpcf_aposent['CO_SEXO_APOSENTADO'][i] == 2 and vpcf_aposent['CO_CONDICAO_APOSENTADO'][i] == 1:
      vpcf_aposent['VALOR_CONTRIB_FUT'][i] = p * (vpcf_aposent['AX_MASC_VALIDOS'][i] * vpcf_aposent['VL_APOSENTADORIA'][i]) * ((vpcf_aposent['VL_APOSENTADORIA'][i] - teto ) / vpcf_aposent['VL_APOSENTADORIA'][i]) * t

    elif vpcf_aposent['CO_SEXO_APOSENTADO'][i] == 1 and vpcf_aposent['CO_CONDICAO_APOSENTADO'][i] == 2:
      vpcf_aposent['VALOR_CONTRIB_FUT'][i] = p * (vpcf_aposent['AX_FEM_INVALIDOS'][i] * vpcf_aposent['VL_APOSENTADORIA'][i]) * ((vpcf_aposent['VL_APOSENTADORIA'][i] - teto ) / vpcf_aposent['VL_APOSENTADORIA'][i]) * t

    elif vpcf_aposent['CO_SEXO_APOSENTADO'][i] == 2 and vpcf_aposent['CO_CONDICAO_APOSENTADO'][i] == 2:
      vpcf_aposent['VALOR_CONTRIB_FUT'][i] = p * (vpcf_aposent['AX_MASC_INVALIDOS'][i] * vpcf_aposent['VL_APOSENTADORIA'][i]) * ((vpcf_aposent['VL_APOSENTADORIA'][i] - teto ) / vpcf_aposent['VL_APOSENTADORIA'][i]) * t 
  else:
    vpcf_aposent['VALOR_CONTRIB_FUT'][i] = 0

<ipython-input-25-6f8ecb3f1884>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vpcf_aposent['VALOR_CONTRIB_FUT'][i] = p * (vpcf_aposent['AX_MASC_VALIDOS'][i] * vpcf_aposent['VL_APOSENTADORIA'][i]) * ((vpcf_aposent['VL_APOSENTADORIA'][i] - teto ) / vpcf_aposent['VL_APOSENTADORIA'][i]) * t
<ipython-input-25-6f8ecb3f1884>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vpcf_aposent['VALOR_CONTRIB_FUT'][i] = p * (vpcf_aposent['AX_FEM_VALIDOS'][i] * vpcf_aposent['VL_APOSENTADORIA'][i]) * ((vpcf_aposent['VL_APOSENTADORIA'][i] - teto ) / vpcf_aposent['VL_APOSENTADORIA'][i]) * t
<ipython-input-25-6f8ecb3f1884>:15: SettingWi

In [26]:
vpcf_aposent['VALOR_CONTRIB_FUT'].sum()

33378465